In this notebook I am going to donwload the data, apply the NER and relationships extraction over the data


# Importing Libraries

In [ ]:
!pip install -U spacy-nightly --pre
!pip install -U pip setuptools wheel
!python -m spacy project clone tutorials/rel_component
!python -m spacy download en_core_web_trf
!pip install -U spacy transformers
!cd rel_component

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 51.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.8 MB/s eta 0:00:00
  Using cached setuptools-68.2.2-py3-none-any.whl (807 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: wheel
    Found existing installation: wheel 0.41.2
    Uninstalling wheel-0.41.2:
      Successfully uninstalled wheel-

2023-11-02 13:21:54.193161: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-02 13:21:54.193222: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-02 13:21:54.193268: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-02 13:21:54.205836: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-02 13:21:55.678271: W tensorflow/compiler/

In [ ]:
!pip install spacy-transformers

  Using cached transformers-4.30.2-py3-none-any.whl.metadata (113 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.14.1
    Uninstalling tokenizers-0.14.1:
      Successfully uninstalled tokenizers-0.14.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.34.1
    Uninstalling transformers-4.34.1:
      Successfully uninstalled transformers-4.34.1


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import spacy
from spacy import displacy
import spacy_transformers

import transformers

import requests
import json
import hashlib

# Importing Dataset
I am going to download the data using world bank api

In [ ]:
url = 'https://search.worldbank.org/api/v2/wds'
params = {
    'format': 'json',
    'display_title': '"sustainable development"',
    'rows': 20,
    'page': 1
}

In this step I am creating a metadata that fetches the result using requests
load them into json file and then as a pandas dataframe

In [ ]:
metadata_list = []

for i in range(1):
    response = requests.get(url, params=params)
    data = json.loads(response.content)
    for doc_id in data['documents']:
        metadata = data['documents'][doc_id]
        metadata_list.append(metadata)

    params['page'] += 1

df = pd.DataFrame(metadata_list)

In [ ]:
df.head()

,id,last_modified_date,admreg,admreg_key,authors,count,count_key,docna,docty,docty_key,...,origu,author_keys,available_in,fullavailablein,url,volnb,abstracts,topicv3,trustfund,trustfund_key
0,34188841,2023-10-31T00:00:00Z,"Latin America & Caribbean,Latin America & Cari...","119228,119228",{'0': {'author': 'ELAINY CRISTINA PINHEIRO VIE...,Brazil,82649,{'0': {'docna': 'Brazil - LATIN AMERICA AND CA...,Procurement Plan,739941,...,Agriculture and Food PG LAC (SLCAG),{'0': {'author_key': 'NA'}},English,[],http://documents.worldbank.org/curated/en/0991...,NaN,NaN,NaN,NaN,NaN
1,34188843,2023-10-31T00:00:00Z,"Latin America & Caribbean,Latin America & Cari...","119228,119228",{'0': {'author': 'Lafaete Almeida de Oliveira'}},Brazil,82649,{'0': {'docna': 'Brazil - LATIN AMERICA AND CA...,Procurement Plan,739941,...,Agriculture and Food PG LAC (SLCAG),{'0': {'author_key': 'NA'}},English,[],http://documents.worldbank.org/curated/en/0991...,NaN,NaN,NaN,NaN,NaN
2,34188148,2023-10-30T00:00:00Z,"Europe and Central Asia,Europe and Central Asia","119226,119226",{'0': {'author': 'zgen alar'}},Turkiye,82578,{'0': {'docna': 'Turkiye - EUROPE AND CENTRAL ...,Procurement Plan,739941,...,Urban ECA (SCAUR),{'0': {'author_key': 'NA'}},English,[],http://documents.worldbank.org/curated/en/0991...,NaN,NaN,NaN,NaN,NaN
3,34186339,2023-10-26T00:00:00Z,"Latin America & Caribbean,Latin America & Cari...","119228,119228","{'0': {'author': 'Santos Lima,Eirivelthon'}}",Colombia,82629,{'0': {'docna': 'Concept Environmental and Soc...,Environmental and Social Review Summary,1661364,...,LAC ENR PM (SLCEN),{'0': {'author_key': '2040295'}},English,[],http://documents.worldbank.org/curated/en/0991...,NaN,NaN,NaN,NaN,NaN
4,34182201,2023-10-18T00:00:00Z,"Latin America & Caribbean,Latin America & Cari...","119228,119228","{'0': {'author': 'Noronha Farinelli,Barbara Cr...",Brazil,82649,{'0': {'docna': 'Environmental and Social Comm...,Environmental and Social Commitment Plan,1680570,...,Agriculture and Food PG LAC (SLCAG),{'0': {'author_key': '1016863'}},English,[],http://documents.worldbank.org/curated/en/0991...,NaN,NaN,NaN,NaN,NaN


This is being created as a dataframe now we need to convert it to documents for NER and the relationship extractions

In [ ]:
def get_all_documents(df):
  '''A function that takes dataframe as input and then convert it to a document format and return it'''

  documents = []#an empty list to store these values with the rows and columns
  for index,row in df.iterrows():
    documents.append(str(row[0]))
  return documents

In [ ]:
documents = get_all_documents(df)
documents = documents[:]

In [ ]:
print(documents)

['34188841', '34188843', '34188148', '34186339', '34182201', '34182199', '34182200', '34178366', '34072674', '34179126', '34176124', '34170188', '34167168', '34167164', '34167167', '34167170', '34164750', '34164262', '34162034', '34162035', 'nan']


# Extracting Entities

In [ ]:
nlp = spacy.load("en_core_web_sm")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
def extract_ents(documents,nlp):
  '''A function for extracting entities that takes the documents and nlp pipeline as input that first get the keys and their respective annotations
  Encdoing them to utf-8 using hashlib  then '''
  docs = list()
  for doc in nlp.pipe(documents, disable=["tagger", "parser"]):
      dictionary=dict.fromkeys(["text", "annotations"])
      dictionary["text"]= str(doc)
      dictionary['text_sha256'] =  hashlib.sha256(dictionary["text"].encode('utf-8')).hexdigest()
      annotations=[]

      for e in doc.ents:
        ent_id = hashlib.sha256(str(e.text).encode('utf-8')).hexdigest()
        ent = {"start":e.start_char,"end":e.end_char, "label":e.label_,"label_upper":e.label_.upper(),"text":e.text,"id":ent_id}
        if e.label_ == "Count":
          ent["count"] = int(e.text[0])

        annotations.append(ent)

      dictionary["annotations"] = annotations
      docs.append(dictionary)
  #print(annotations)
  return docs

In [ ]:
parsed_ents = extract_ents(documents,nlp)

/usr/local/lib/python3.10/dist-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [ ]:
parsed_ents

[{'text': '34188841',
  'annotations': [{'start': 0,
    'end': 8,
    'label': 'CARDINAL',
    'label_upper': 'CARDINAL',
    'text': '34188841',
    'id': '871f7277fa7cdaaf69142f909f22378e00eac3064712fad1ed21f52466a96369'}],
  'text_sha256': '871f7277fa7cdaaf69142f909f22378e00eac3064712fad1ed21f52466a96369'},
 {'text': '34188843',
  'annotations': [{'start': 0,
    'end': 8,
    'label': 'DATE',
    'label_upper': 'DATE',
    'text': '34188843',
    'id': '8f79fa2b2e5b69b6cd19401c5c4431f926752dde5be3e24569684325b9a6db1d'}],
  'text_sha256': '8f79fa2b2e5b69b6cd19401c5c4431f926752dde5be3e24569684325b9a6db1d'},
 {'text': '34188148',
  'annotations': [{'start': 0,
    'end': 8,
    'label': 'DATE',
    'label_upper': 'DATE',
    'text': '34188148',
    'id': 'da791788755f1ddcce7a77abd17a0b5ebb9d746c7ddbf9525e28fa71a1fc6223'}],
  'text_sha256': 'da791788755f1ddcce7a77abd17a0b5ebb9d746c7ddbf9525e28fa71a1fc6223'},
 {'text': '34186339',
  'annotations': [{'start': 0,
    'end': 8,
    'label

# Extracting Relationships

Here I am loading the roberta spacy model to extarct out the relationships

In [ ]:
# %cd rel_component/
import random
import typer
from pathlib import Path
import spacy
from spacy.tokens import DocBin, Doc
from spacy.training.example import Example

# make the factory work
from scripts.rel_pipe import make_relation_extractor, score_relations

# make the config work
from scripts.rel_model import create_relation_model, create_classification_layer, create_instances, create_tensors

In [ ]:
!python3 -m spacy download en_core_web_trf

2023-11-02 13:23:37.106643: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-02 13:23:37.106713: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-02 13:23:37.106755: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-02 13:23:39.041071: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━ 191.7/457.4 MB 67.4 MB/s eta 0:00:04
ERROR: Wheel 'en-core-web-trf' located at /tmp/pip-unpack-97x3zn5y/en_core_web_trf-3.7.2-py3-none-any.whl is invalid.


In [ ]:
# Load the spacy transformer (roberta-base) model
roberta_nlp = spacy.load("en_core_web_trf",disable=["tagger", "ner", "lemmatizer"])

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_trf' (3.6.1) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
def extract_relations(documents,nlp,nlp2):
  predicted_rels = list()
  for doc in nlp.pipe(documents, disable=["tagger", "parser"]):
    source_hash = hashlib.sha256(doc.text.encode('utf-8')).hexdigest()
    for name, proc in nlp2.pipeline:
          doc = proc(doc)

    for value, rel_dict in doc._.rel.items():
      for e in doc.ents:
        for b in doc.ents:
          if e.start == value[0] and b.start == value[1]:
            max_key = max(rel_dict, key=rel_dict. get)
            print(max_key)
            e_id = hashlib.sha256(str(e).encode('utf-8')).hexdigest()
            b_id = hashlib.sha256(str(b).encode('utf-8')).hexdigest()
            if rel_dict[max_key] >=0.9 :
              print(f" entities: {e.text, b.text} --> predicted relation: {rel_dict}")
              predicted_rels.append({'head': e_id, 'tail': b_id, 'type':max_key, 'source': source_hash})
  return predicted_rels

In [ ]:
predicted_rels = extract_relations(documents,nlp,roberta_nlp)

In [ ]:
def return_entities(pipeline, texts):
    # List to store entity text and label pairs
    entities_list = []
    document = pipeline(texts)

    # Entity text & label extraction
    for entity in document.ents:
        entities_list.append((entity.text, entity.label_))

    # You may also want to render each document's entities using displacy
    displacy.render(document, jupyter=True, style='ent')

    return entities_list

In [ ]:
print(documents)

['34188841', '34188843', '34188148', '34186339', '34182201', '34182199', '34182200', '34178366', '34072674', '34179126', '34176124', '34170188', '34167168', '34167164', '34167167', '34167170', '34164750', '34164262', '34162034', '34162035', 'nan']


In [ ]:
df_str = df.to_string()

In [ ]:
list_entities=return_entities(roberta_nlp,df_str)

/usr/local/lib/python3.10/dist-packages/spacy/displacy/__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [55]:
url = "https://documents1.worldbank.org/curated/en/099092823161580577/text/BOSIB055c2cb6c006090a90150e512e6beb.txt"
response = requests.get(url)

if response.status_code == 200:
    with open("downloaded_file.txt", "w", encoding="utf-8") as file:
        file.write(response.text)
    print("File downloaded successfully.")
else:
    print(f"Failed to download file. Status code: {response.status_code}")

File downloaded successfully.


In [56]:
file_path = "downloaded_file.txt"

with open(file_path, "r", encoding="utf-8") as file:
    file_content = file.read()

# Now, 'file_content' contains the content of the file
print(file_content)

A New Era
in Development
ANNUAL REPORT 2023
Contents
	 3	Introduction
	 4	 Message from the President
	 6	 Message from the Executive Directors
	 9	 Addressing a Multitude of Global Challenges
	 15	 Regional Engagements
	45	 Working to Achieve Development Goals
	 71	 Promoting Sustainable Finance and Capital Markets
	 75	 Engaging with Partners on Shared Priorities
	 78	 Enhancing Effectiveness and Sustainability in Our Operations
	 81 	 Our Values, People, and Places
	 87	 The World Bank’s Climate-Related Disclosures
	89	 Guiding the Institution
	90 	 Providing Oversight and Accountability
	94 	 Deploying Resources Strategically
	106 	 Commitment to Results

KEY TABLES
	
100 	IBRD Key Financial Indicators, Fiscal 2019–23
	
104 	IDA Key Financial Indicators, Fiscal 2019–23


This annual report, which covers the period from July 1, 2022, to June 30, 2023,
has been prepared by the Executive Directors of both the International Bank for
Reconstruction and Development (IBRD) and the Inter